In [1]:
#Make_SUS_Airedale_FDM_Format.ipynb
##last modified 20230309 

#Builds the BDCT SUS FDM
#Assumes that the targetdb below exists
#Needs code to build it if it doesn't

#You will need to manually amend the target dataset for this script

library(bigrquery)
library(lubridate) 

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

sourcedb <-"yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"
targetdb <-'yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_SUS_Airedale_FDM_Format'
lookups <-'yhcr-prd-phm-bia-core.CB_LOOKUPS'
targetdb <-gsub(' ','',targetdb)
print (sourcedb) 
print (targetdb)
print(lookups)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




[1] "yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"
[1] "yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_SUS_Airedale_FDM_Format"
[1] "yhcr-prd-phm-bia-core.CB_LOOKUPS"


In [2]:
# Set the date fro this to the last date you downloaded the data 
# Ensure fdm_data_source date has been updated

sql14 <- 'update yhcr-prd-phm-bia-core.CB_FDM_Management.cb_source_data_list set updated_date = \'2023-10-16\' 
where dataset = "CB_STAGING_DATABASE_SUS_Airedale_FDM_Format";' 
#print(sql14)
tb14 <- bq_project_query(project_id, sql14)


eric <- paste("fdm_source_date updated "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 


[1] "fdm_source_date updated 2023-10-19 17:07:29.56056"


In [3]:
sql4 <-paste('drop table if exists ',targetdb,'.tbl_SUS_Airedale_APC_20220601_to_Current;', sep = "")
tb4 <-bq_project_query(project_id, sql4)



sql5 <-paste('drop table if exists ',targetdb,'.tbl_SUS_Airedale_OP_20220601_to_Current;', sep = "")
tb5 <-bq_project_query(project_id, sql5)

sql6 <-paste('drop table if exists ',targetdb,'.tbl_SUS_Airedale_ECDS_20220601_to_Current;', sep = "")
tb6 <-bq_project_query(project_id, sql6)


eric <- paste("tables deleted from target "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 


[1] "tables deleted from target 2023-10-19 17:07:35.309175"


In [4]:
sql8 <-paste('create or replace TABLE ', targetdb , '.tbl_SUS_Airedale_APC_20220601_to_Current 
as 
SELECT 
dem.person_id,
cast(left(admdate,4)||\'-\'||substring(admdate,5,2)||\'-\'|| substring(admdate,7,2)||\' 00:00:00\' as datetime) as tbl_SUS_Airedale_APC_20220601_to_Current_start_date,
cast(left(dischdate,4)||\'-\'||substring(dischdate,5,2)||\'-\'|| substring(dischdate,7,2)||\' 00:00:00\' as datetime)  as tbl_SUS_Airedale_APC_20220601_to_Current_end_date,
src.* FROM ', sourcedb,'.src_SUS_Airedale_APC_20220601_to_Current src 
left join yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER dem
on src.digest = dem.digest ;', sep = "")
tb8 <-bq_project_query(project_id, sql8)


eric <- paste("APC Built "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 


[1] "APC Built 2023-10-19 17:07:48.532596"


In [5]:
sql9<-paste('create or replace TABLE ', targetdb , '.tbl_SUS_Airedale_ECDS_20220601_to_Current 
as 
SELECT 
dem.person_id,
cast(arrival_date ||\' \' || arrival_time as datetime) as tbl_SUS_Airedale_ECDS_20220601_to_Current_start_date,
cast(depart_date ||\' \' || depart_time as datetime) as tbl_SUS_Airedale_ECDS_20220601_to_Current_end_date,
src.* FROM yhcr-prd-phm-bia-core.CB_MYSPACE_JDB.src_SUS_Airedale_ECDS_20220601_to_Current src
 left join  yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER dem
on src.digest = dem.digest
;', sep = "")
tb9 <-bq_project_query(project_id, sql9)


eric <- paste("ECDS Built "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 


[1] "ECDS Built 2023-10-19 17:08:00.59156"


In [6]:
sql10<-paste('create or replace TABLE ', targetdb , '.tbl_SUS_Airedale_OP_20220601_to_Current 
as 
SELECT 
dem.person_id,
cast(left(appdate,4)||\'-\'||substring(appdate,5,2)||\'-\'|| substring(appdate,7,2)||\' \'|| substring(apptime,1,2)||\':\'|| substring(apptime,3,2)||\':00\'  as datetime) as tbl_SUS_Airedale_OP_20220601_to_Current_start_date,
cast(left(appdate,4)||\'-\'||substring(appdate,5,2)||\'-\'|| substring(appdate,7,2)||\' \'|| substring(apptime,1,2)||\':\'|| substring(apptime,3,2)||\':00\'  as datetime) as tbl_SUS_Airedale_OP_20220601_to_Current_end_date,
src.* FROM yhcr-prd-phm-bia-core.CB_MYSPACE_JDB.src_SUS_Airedale_OP_20220601_to_Current src 
 left join  `yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER` dem
on src.digest = dem.digest
;', sep = "")
tb10 <-bq_project_query(project_id, sql10)

eric <- paste("OP Built "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

[1] "OP Built 2023-10-19 17:08:13.98532"
